## Test accuracy on 20 CIFAR-100 classes using ResNet50V2

### Import all the necessary libraries

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.datasets import cifar100

# 1. List GPUs and set memory growth & limit BEFORE any TF initialization
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Enable memory growtha
            tf.config.experimental.set_memory_growth(gpu, True)
        # Optional: set a memory limit (here 8GB)
        tf.config.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8192)])
    except RuntimeError as e:
        print("RuntimeError:", e)

# 2. Check available devices and memory limits
print("Available devices and their memory limits:")
for device in device_lib.list_local_devices():
    print(f"{device.name} — {device.device_type} — {device.memory_limit / 1024 / 1024:.2f} MiB")


2025-05-26 23:43:09.117289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748281389.252674    7388 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748281389.289589    7388 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748281389.607477    7388 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748281389.607497    7388 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748281389.607499    7388 computation_placer.cc:177] computation placer alr

AttributeError: module 'tensorflow._api.v2.config' has no attribute 'set_virtual_device_configuration'

###  Load CIFAR-100 fine labels (100-class)

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

### Choose 20 classes

In [ ]:
selected_classes = list(range(20))
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

x_train_20 = x_train[train_mask]
y_train_20 = y_train[train_mask]
x_test_20 = x_test[test_mask]
y_test_20 = y_test[test_mask]

### Map label to 0 - 19

In [ ]:
label_map = {cls: i for i, cls in enumerate(selected_classes)}
y_train_20 = np.array([label_map[y[0]] for y in y_train_20])
y_test_20 = np.array([label_map[y[0]] for y in y_test_20])

### Resize to 224x224

In [ ]:
x_train_20 = tf.image.resize(x_train_20, (128, 128)).numpy()
x_test_20 = tf.image.resize(x_test_20, (128, 128)).numpy()

2025-05-26 23:33:39.493211: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.0KiB (rounded to 1280)requested by op ScratchBuffer
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-05-26 23:33:39.493241: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-05-26 23:33:39.493247: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-05-26 23:33:39.493250: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2025-05-26 23:33:39.493253: I external/l

FailedPreconditionError: {{function_node __wrapped____EagerConst_device_/job:localhost/replica:0/task:0/device:GPU:0}} Failed to allocate scratch buffer for device 0

122] 1 Chunks of size 47251456 totalling 45.06MiB
2025-05-26 23:33:39.493342: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1126] Sum Total of in-use chunks: 45.06MiB
2025-05-26 23:33:39.493344: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1128] Total bytes in pool: 47251456 memory_limit_: 47251456 available bytes: 0 curr_region_allocation_bytes_: 94502912
2025-05-26 23:33:39.493349: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1133] Stats: 
Limit:                        47251456
InUse:                        47251456
MaxInUse:                     47251456
NumAllocs:                           4
MaxAllocSize:                 47251456
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2025-05-26 23:33:39.493352: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:512] ******************************************************************xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
2025-05-26 23:33:39

### Preprocess

In [ ]:
x_train_20 = preprocess_input(x_train_20)
x_test_20 = preprocess_input(x_test_20)

### Converts Label

In [ ]:
y_train_20_cat = tf.keras.utils.to_categorical(y_train_20, 20)
y_test_20_cat = tf.keras.utils.to_categorical(y_test_20, 20)

### Load ResNet50V2 base model

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(20, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train_20, y_train_20_cat, epochs=5, batch_size=64, validation_split=0.1)

loss, acc = model.evaluate(x_test_20, y_test_20_cat)
print("Test accuracy on 20 CIFAR-100 classes using ResNet50:", acc)

Epoch 1/5


2025-05-25 02:41:57.363303: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1769472000 exceeds 10% of free system memory.


141/141 [==============================] - 129s 900ms/step - loss: 0.9260 - accuracy: 0.7208 - val_loss: 0.5222 - val_accuracy: 0.8240
Epoch 2/5
141/141 [==============================] - 123s 876ms/step - loss: 0.4528 - accuracy: 0.8481 - val_loss: 0.5207 - val_accuracy: 0.8220
Epoch 3/5
141/141 [==============================] - 122s 868ms/step - loss: 0.3285 - accuracy: 0.8858 - val_loss: 0.5402 - val_accuracy: 0.8220
Epoch 4/5
141/141 [==============================] - 122s 868ms/step - loss: 0.2539 - accuracy: 0.9122 - val_loss: 0.5287 - val_accuracy: 0.8350
Epoch 5/5
63/63 [==============================] - 23s 358ms/step - loss: 0.4656 - accuracy: 0.8595
Test accuracy on 20 CIFAR-100 classes using ResNet50: 0.859499990940094
